In [68]:
#! import all necessary lib
from PIL import Image
import numpy as np
import csv


# CONVERT MAP RECEIVE TO BINARY MATRIX

In [69]:

image = Image.open("../rawData/Map19_12.pgm")
image.save("../map/Map19_12.png")

In [70]:
#! convert map image to matrix binary


image_path = "../map/Map19_12.png"
image = Image.open(image_path).convert("RGB")

image_arr = np.array(image)

def convertToMatrix(image_arr):
    matrix = np.zeros((image_arr.shape[0], image_arr.shape[1]), dtype=int)
    
    #! map white pixels to 255
    whiteMask = (image_arr == [255,255,255]).all(axis=-1) | (image_arr == [254,254,254]).all(axis = -1)
    matrix[whiteMask] = 1
    
    return matrix

matrix = convertToMatrix(image_arr)
outputCsvFile ="../shortestPathResult/dataCSV/map1912.csv"

with open(outputCsvFile, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(matrix)
print(f"matrix have saved in path: {outputCsvFile}")


matrix have saved in path: ../shortestPathResult/dataCSV/map1912.csv


In [71]:


# Load the matrix from a CSV file
def load_matrix_from_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        matrix = []
        for row in reader:
            # Convert numeric values and leave 'X' as is
            matrix.append([int(cell) if cell.isdigit() else cell for cell in row])
    return matrix

# Convert the matrix to a PNG image
def matrix_to_png(matrix, output_file):
    height = len(matrix)
    width = len(matrix[0])

    # Create a new RGB image
    image = Image.new("RGB", (width, height))

    # Map matrix values to RGB
    pixels = image.load()
    for y in range(height):
        for x in range(width):
            if matrix[y][x] == 0:  # Obstacle
                pixels[x, y] = (0, 0, 0)  # Black
            elif matrix[y][x] == 1:  # Free space
                pixels[x, y] = (255, 255, 255)  # White
            elif matrix[y][x] == 'X':  # Robot path
                pixels[x, y] = (255, 255, 0)  # Yellow

    # Save the image
    image.save(output_file)
    print(f"PNG file saved as '{output_file}'")

# Main function
def convertFromCSVToPNG(yourInputCSVPath: str,yourAlgor: str):
    # Path to your CSV file
    input_csv = yourInputCSVPath

    # Output PNG file
    output_png = "../shortestPathResult/"+yourAlgor+".png"

    # Load the matrix and convert it to PNG
    matrix = load_matrix_from_csv(input_csv)
    matrix_to_png(matrix, output_png)




# PREPROCESS DATA

In [72]:

def loadMatrix(path):
    with open(path, mode="r") as file:
        reader = csv.reader(file)
        matrix = [list(map(int, row)) for row in reader]
    return matrix
def markPath(matrix, path):
    for (x,y) in path:
        matrix[x][y] = "X"
    return matrix
def getPoint(matrix):
    rows, cols = len(matrix), len(matrix[0])
    travelPoint = [(i,j) for i in range(rows) for j in range(cols) if matrix[i][j] == 1]
    if not travelPoint:
        return None, None
    
    def distance(p1,p2):
        import math
        return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 )
    
    start = min(travelPoint, key=lambda p: distance((0,0),p))
    end = max(travelPoint, key=lambda p: distance((0,0),p))
    return start,end
def saveResult(matrix, path):
    with open(path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(matrix)
    print(f"Result has been saved to path: {path}")

# DFS

In [73]:
def DFS(matrix, start: set, end: set):
    rows, cols = len(matrix), len(matrix[0])
    stack = [start]
    visited = set()
    parent = {} #! for truy vet the path
    
    while stack:
        x,y = stack.pop()
        if (x,y) == end: #! termination
            path = []
            while (x,y) in parent:
                path.append((x,y))
                x,y = parent[(x,y)]
                
            path.append(start)
            return path[::-1]
        
        #! move robot
        if (x,y) not in visited:
            visited.add((x,y))
            
            for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                nx, ny = x + dx, y+dy
                if (0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1 and (nx,ny) not in visited):
                    stack.append((nx,ny))
                    parent[(nx,ny)] = (x,y)
    #! termination for no path found
    return None

# BFS

In [74]:

def BFS(matrix, start, end):
    from collections import deque
    rows, cols = len(matrix), len(matrix[0])
    queue = deque()
    queue.append(start)
    visited = set()
    parent = {}
    
    while queue:
        x,y = queue[0]
        queue.popleft()
        if (x,y) == end:
            path = []
            while(x,y) in parent:
                path.append((x,y))   
                x,y = parent[(x,y)]
            path.append(start)
            return path[::-1]
        if (x,y) not in visited:
            visited.add((x,y))
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1 and (nx, ny) not in visited:
                    queue.append((nx, ny))
                    parent[(nx, ny)] = (x, y)  # Track the parent for path reconstruction
    return None

# Astar Algorithm

In [75]:

def aStar(matrix, start: set, end: set):
    import heapq #! working as priority queue
    rows, cols = len(matrix), len(matrix[0])
    
    openList = []
    heapq.heappush(openList,(0,start)) #! (f, (x,y))
    gCost = {start: 0}
    parent = {}
    
    def heuristic(a,b):
        '''
        return mahatan distance
        '''
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    while openList:
        _, current = heapq.heappop(openList)
        x,y = current
        
        if current == end:
            path = []
            while current in parent:
                path.append(current)
                current = parent[current]
            path.append(start)
            return path[::-1]
    
        for dx, dy in [(-1,0), (1,0), (0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1:
                #print(f"current start ({nx},{ny})")
                gNew = gCost[(x,y)] + 1
                if (nx, ny) not in gCost or gNew < gCost[(nx, ny)]:
                    gCost[(nx,ny)] = gNew
                    fCost = gNew + heuristic((nx,ny), end)
                    heapq.heappush(openList, (fCost, (nx,ny)))
                    parent[(nx,ny)] = (x,y)
    return None

# Hill Climbing algorithm

In [76]:
def HillClimb(matrix, start,end):
    rows, cols = len(matrix), len(matrix[0])
    
    def heuristic(p1,p2):
        #print(f"this is p1: {p1}")
        #print(f"this is p2 {p2}")
        return abs(p2[0] - p1[0]) + abs(p2[1] - p1[1])
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    curr = start
    if isinstance(curr,int):
        raise ValueError("Start should be a set of tuple (x,y)")
    currPath = [curr]
    visited[curr[0]][curr[1]] = True
    while(curr != end):
        x,y = curr
        
            
        neighbour = []
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x+dx, y+dy
            if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 1:
                neighbour.append((heuristic((nx,ny), end),(nx,ny)))
        if not neighbour:
            print("stuck at local optimum")
            return currPath
        neighbour.sort(key=lambda x: x[0])
        bestNeighbour = neighbour[0][1]
        
        if heuristic(bestNeighbour, end) >= heuristic(curr,end):
            print("cant find better neighbour, stuck at local minimum")
            return currPath
        
        curr = bestNeighbour
        currPath.append(curr)
        visited[curr[0]][curr[1]] = True
    return currPath

# Genetic algorithm

In [77]:
# import random
# random.seed(1234)
# def Genetic(matrix, start, end, populationSize = 100, generations = 200, mutationRate = 0.1):
#     rows, cols = len(matrix), len(matrix[0])
    
#     def heuristic(p1,p2):
#         return abs(p2[0] - p1[0]) + abs(p2[1] - p1[1])
    
#     def generateIndividual():
#         path = [start]
#         while(path[-1] != end):
#             x,y = path[-1]
#             neighbour = [(x + dx, y + dy) for dx,dy in [(-1,0),(0,-1),(1,0),(0,1)] if 0 <= x + dx < rows and 0 <= y + dy < cols and matrix[x + dx][y + dy] == 1]
#             if not neighbour:
#                 break #! no valid neibour then stop
            
#             path.append(random.choice(neighbour))
#         return path
    
#     def fitness(individual):
#         '''
#         calculate fitness based on path length and reaching the goal
#         '''
#         if individual[-1] != end:
#             return float('inf')
#         return len(individual) + heuristic(individual[-1],end)
    
#     def crossOver(parent1, parent2):
#         split = random.randint(1,min(len(parent1), len(parent2))-1)
#         child = parent1[:split]
        
#         for point in parent2:
#             if point not in child:
#                 child.append(point)
#             if point == end:
#                 break
#         return child
    
#     def mutate(individual):
#         '''
#         mutate an individual by altering a random segment
#         '''
#         if random.random() < mutationRate:
#             idx = random.randint(1, len(individual) - 2) #! avoid mutating start or end
#             x,y = individual[idx]
#             neighbour = [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
#                          if 0 <= x + dx < rows and 0 <= y + dy < cols and matrix[x + dx][y + dy] == 1]
#             if neighbour:
#                 individual[idx] = random.choice(neighbour)
#         return individual
#     population = [generateIndividual() for _ in range(populationSize)]
    
#     for generation in range(generations):
#         population.sort(key=fitness)
#         if fitness(population[0]) == len(population[0]):
#             print(f"solution found at generation {generation}")
#             return population[0]
#         #! selection: keep the top 50%
#         topIndividual = population[:populationSize]
#         #! cross over and mutation
#         newPopulation = []
#         for _ in range(populationSize):
#             parent1, parent2 = random.sample(topIndividual, 2)
#             child = crossOver(parent1,parent2)
#             child = mutate(child)
#             newPopulation.append(child)
#         population = newPopulation
#     print("No solution found!")
#     return population[0]
import random
random.seed(1234)

def Genetic(matrix, start, end, populationSize=100, generations=200, mutationRate=0.1):
    rows, cols = len(matrix), len(matrix[0])
    
    def heuristic(p1, p2):
        return abs(p2[0] - p1[0]) + abs(p2[1] - p1[1])
    
    def generateIndividual():
        path = [start]
        while(path[-1] != end):
            x, y = path[-1]
            neighbour = [(x + dx, y + dy) for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)] 
                        if 0 <= x + dx < rows and 0 <= y + dy < cols and matrix[x + dx][y + dy] == 1]
            if not neighbour:
                break  # no valid neighbour, then stop
            
            path.append(random.choice(neighbour))
        return path
    
    def fitness(individual):
        '''
        Calculate fitness based on path length and reaching the goal.
        '''
        if individual[-1] != end:
            return float('inf')
        return len(individual) + heuristic(individual[-1], end)
    
    def crossOver(parent1, parent2):
        # Ensure both parents are long enough for crossover
        min_length = min(len(parent1), len(parent2))
        
        # Avoid crossover if either parent is too short
        if min_length <= 1:
            return parent1  # Or return a modified version if desired
        
        split = random.randint(1, min_length - 1)  # This ensures the range is valid
        child = parent1[:split]
        
        for point in parent2:
            if point not in child:
                child.append(point)
            if point == end:
                break
        return child
    
    def mutate(individual):
        '''
        Mutate an individual by altering a random segment.
        '''
        if random.random() < mutationRate:
            # Check if the individual is long enough to mutate
            if len(individual) > 2:  # Ensure the length is greater than 2 to mutate
                idx = random.randint(1, len(individual) - 2)  # Avoid mutating start or end
                x, y = individual[idx]
                neighbour = [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
                            if 0 <= x + dx < rows and 0 <= y + dy < cols and matrix[x + dx][y + dy] == 1]
                if neighbour:
                    individual[idx] = random.choice(neighbour)
        return individual

    # Initialize the population
    population = [generateIndividual() for _ in range(populationSize)]
    
    for generation in range(generations):
        population.sort(key=fitness)
        if fitness(population[0]) == len(population[0]):
            print(f"Solution found at generation {generation}")
            return population[0]
        
        # Selection: Keep the top 50%
        topIndividual = population[:populationSize // 2]
        
        # Crossover and mutation
        newPopulation = []
        for _ in range(populationSize):
            parent1, parent2 = random.sample(topIndividual, k=2)  # Fix: specify k=2 for selection
            child = crossOver(parent1, parent2)
            child = mutate(child)
            newPopulation.append(child)
        
        population = newPopulation
    
    print("No solution found!")
    return population[0]


# UNIFORM COST SEARCH

In [78]:
import heapq

def ucs(matrix, startPoint, endPoint):
    """
    Uniform Cost Search (UCS) algorithm for a grid-based matrix.
    
    Args:
        matrix (list of list of int): The map where 1 = walkable, 0 = obstacle.
        startPoint (tuple): The starting point as (row, col).
        endPoint (tuple): The ending point as (row, col).
    
    Returns:
        list: The shortest path from startPoint to endPoint as a list of (row, col) tuples.
              Returns an empty list if no path exists.
    """
    rows, cols = len(matrix), len(matrix[0])
    
    # Priority queue for UCS (cost, (row, col), path)
    queue = [(0, startPoint, [startPoint])]
    visited = set()  # To keep track of visited nodes
    
    # Directions for moving up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    while queue:
        # Get the node with the smallest cost
        cost, (row, col), path = heapq.heappop(queue)
        
        # If we reached the goal, return the path
        if (row, col) == endPoint:
            return path
        
        # Skip if the cell is already visited
        if (row, col) in visited:
            continue
        visited.add((row, col))
        
        # Explore neighbors
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            
            # Check bounds and if the cell is walkable
            if 0 <= new_row < rows and 0 <= new_col < cols and matrix[new_row][new_col] == 1:
                if (new_row, new_col) not in visited:
                    # Add neighbor to the queue with updated cost
                    heapq.heappush(queue, (cost + 1, (new_row, new_col), path + [(new_row, new_col)]))
    
    # If we exhaust the queue without finding the goal, return an empty list
    return []


# CONVERT FROM MATRIX CSV TO IMAGE

# MAIN FUNCTION

## LOAD MATRIX

In [79]:
#! main function to run all algorithm
MAPPATH = "../shortestPathResult/dataCSV/map1912.csv"
matrix = loadMatrix(MAPPATH)

In [80]:
allAlgorName = ["DFS", "BFS", "aStar", "HillClimb", "Genetic", "ucs"]
theEndPath = "path.csv"
savePath = "../shortestPathResult/"
allResultPath = [savePath + allAlgorName[i] + theEndPath for i in range(len(allAlgorName))]


In [81]:
allResultPath

['../shortestPathResult/DFSpath.csv',
 '../shortestPathResult/BFSpath.csv',
 '../shortestPathResult/aStarpath.csv',
 '../shortestPathResult/HillClimbpath.csv',
 '../shortestPathResult/Geneticpath.csv',
 '../shortestPathResult/ucspath.csv']

In [ ]:

# start, end = getPoint(matrix)
# print(f"start point: {start}")
# print(f"end point: {end}")
start = (40,60)
end = (80,80)
for i in range(len(allResultPath)):
    if allAlgorName[i] == "DFS":
        path = DFS(matrix,start,end)
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrix, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "BFS":
        path = BFS(matrix,start,end)
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrix, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "aStar":
        path = aStar(matrix,start,end)
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrix, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "Genetic":
        path = Genetic(matrix,start,end)
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrix, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    if allAlgorName[i] == "HillClimb":
        path = HillClimb(matrix,start,end)
        if path:
            print(f"Path found for {allAlgorName[i]} algorithm!: ")
            print(path)
            matrixWithPath = markPath(matrix, path)
            
            saveResult(matrix, allResultPath[i])
            convertFromCSVToPNG(allResultPath[i],allAlgorName[i])
        else:
            print(f"No path exist between {start} to {end} for algorithm {allAlgorName[i]}")
    # if i == "DFS":
    #     path = DFS(matrix,start,end)
    # if i == "DFS":
    #     path = DFS(matrix,start,end)
#path = HillClimb(matrix, start,end)

    

No path exist between (40, 60) to (80, 100) for algorithm DFS
No path exist between (40, 60) to (80, 100) for algorithm BFS
No path exist between (40, 60) to (80, 100) for algorithm aStar
cant find better neighbour, stuck at local minimum
Path found for HillClimb algorithm!: 
[(40, 60), (40, 61), (41, 61), (42, 61), (43, 61), (44, 61), (45, 61), (46, 61), (47, 61), (48, 61), (49, 61), (50, 61), (51, 61), (52, 61), (53, 61), (54, 61), (55, 61), (56, 61), (57, 61), (58, 61), (59, 61), (60, 61), (61, 61), (62, 61), (63, 61), (64, 61), (65, 61), (65, 62), (66, 62), (67, 62), (68, 62), (68, 63), (69, 63), (70, 63), (71, 63), (71, 64), (72, 64), (73, 64), (74, 64), (74, 65), (75, 65), (76, 65), (77, 65), (78, 65), (79, 65), (79, 66), (79, 67), (79, 68), (79, 69), (79, 70), (79, 71), (79, 72), (79, 73), (79, 74), (79, 75), (79, 76), (79, 77), (79, 78), (79, 79), (79, 80), (79, 81), (79, 82), (79, 83), (79, 84), (79, 85), (79, 86), (79, 87), (79, 88), (79, 89), (79, 90), (79, 91), (79, 92), (7

KeyboardInterrupt: 